In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50

Intializing ImageDataGenerator which is used to perform all Pre-Processing and Data Augmentation such as Rescaling, Resizing, Flipping and changing orientations etc

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
img_width = 224
img_height = 224
batch_size = 32

Initializing Resnet50 pretrained model without including the fully connected layers as we want our own fully connected layer trained for our dataset.

In [ ]:
conv_base = ResNet50(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

Creating a function which will create 2 arrays one is for extracted features from the last layer before going into the fully connected layer. Another array is there contains the labels for the extracted features.

In [3]:
def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 7, 7, 2048))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count,10))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='categorical')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 1088
validation_size = 256

Passing dataset path and number of observations for training dataset.

In [4]:
train_features, train_labels = extract_features('../input/10-monkey-species/training/training/', train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features('../input/10-monkey-species/validation/validation/', validation_size)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [5]:
features = np.concatenate((train_features, validation_features))

In [6]:
labels_train= []
for i in range(len(train_labels)):
    labels_train.append(np.argmax(train_labels[i])) 
    
labels_valid= []
for i in range(len(validation_labels)):
    labels_valid.append(np.argmax(validation_labels[i])) 

In [7]:
labels = np.concatenate((labels_train, labels_valid))

Reshaping the features 

In [8]:
X_train, y_train = features.reshape(1344,7*7*2048),labels

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
pred = nb.predict(x_test)

In [10]:
accuracy_score(y_test,pred)

0.21189591078066913

## Using InceptionV3

In [11]:
# Extract features
import os, shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
img_width = 224
img_height = 224
batch_size = 32
# Instantiate convolutional base
from keras.applications import InceptionV3

conv_base = InceptionV3(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures
def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 5,5, 2048))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count,10))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='categorical')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 1088
validation_size = 256

87916544/87910968 [==============================] - 3s 0us/step


In [12]:
train_features, train_labels = extract_features('../input/10-monkey-species/training/training/', train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features('../input/10-monkey-species/validation/validation/', validation_size)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [13]:
features = np.concatenate((train_features, validation_features))

In [14]:
labels_train= []
for i in range(len(train_labels)):
    labels_train.append(np.argmax(train_labels[i])) 
    
labels_valid= []
for i in range(len(validation_labels)):
    labels_valid.append(np.argmax(validation_labels[i])) 

In [15]:
labels = np.concatenate((labels_train, labels_valid))

In [16]:
X_train, y_train = features.reshape(1344,5*5*2048), labels

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
pred = nb.predict(x_test)

In [18]:
accuracy_score(y_test,pred)

0.9107806691449815

## Using VGG19

In [19]:
# Extract features
import os, shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
img_width = 224
img_height = 224
batch_size = 32
# Instantiate convolutional base
from keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures
def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 7,7,512))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count,10))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='categorical')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 1088
validation_size = 256

80142336/80134624 [==============================] - 3s 0us/step


In [20]:
train_features, train_labels = extract_features('../input/10-monkey-species/training/training/', train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features('../input/10-monkey-species/validation/validation/', validation_size)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [21]:
features = np.concatenate((train_features, validation_features))

In [22]:
labels_train= []
for i in range(len(train_labels)):
    labels_train.append(np.argmax(train_labels[i])) 
    
labels_valid= []
for i in range(len(validation_labels)):
    labels_valid.append(np.argmax(validation_labels[i])) 

In [23]:
labels = np.concatenate((labels_train, labels_valid))

In [24]:
X_train, y_train = features.reshape(1344,7*7*512), labels

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
pred = nb.predict(x_test)

In [26]:
accuracy_score(y_test,pred)

0.49814126394052044

## Using InceptionResNetV2

In [27]:
# Extract features
import os, shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
img_width = 224
img_height = 224
batch_size = 32
# Instantiate convolutional base
from keras.applications import InceptionResNetV2


conv_base = InceptionResNetV2(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 5,5,1536))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count,10))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='categorical')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 1088
validation_size = 256

219062272/219055592 [==============================] - 8s 0us/step


In [28]:
train_features, train_labels = extract_features('../input/10-monkey-species/training/training/', train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features('../input/10-monkey-species/validation/validation/', validation_size)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [29]:
features = np.concatenate((train_features, validation_features))

In [30]:
labels_train= []
for i in range(len(train_labels)):
    labels_train.append(np.argmax(train_labels[i])) 
    
labels_valid= []
for i in range(len(validation_labels)):
    labels_valid.append(np.argmax(validation_labels[i])) 

In [31]:
labels = np.concatenate((labels_train, labels_valid))

In [32]:
X_train, y_train = features.reshape(1344,5*5*1536), labels

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
pred = nb.predict(x_test)

In [34]:
accuracy_score(y_test,pred)

0.9144981412639405

## Using MobileNetV2

In [35]:
# Extract features
import os, shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
img_width = 224
img_height = 224
batch_size = 32
# Instantiate convolutional base
from keras.applications import MobileNetV2


conv_base = MobileNetV2(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 7,7,1280))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count,10))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='categorical')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 1088
validation_size = 256

9412608/9406464 [==============================] - 1s 0us/step


In [36]:
train_features, train_labels = extract_features('../input/10-monkey-species/training/training/', train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features('../input/10-monkey-species/validation/validation/', validation_size)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [37]:
features = np.concatenate((train_features, validation_features))

In [38]:
labels_train= []
for i in range(len(train_labels)):
    labels_train.append(np.argmax(train_labels[i])) 
    
labels_valid= []
for i in range(len(validation_labels)):
    labels_valid.append(np.argmax(validation_labels[i])) 

In [39]:
labels = np.concatenate((labels_train, labels_valid))

In [40]:
X_train, y_train = features.reshape(1344,7*7*1280), labels

x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [41]:
pred = nb.predict(x_test)

In [42]:
accuracy_score(y_test,pred)

0.9256505576208178